In [6]:
import matplotlib.pyplot as plt
import torchvision
import random 
import numpy as np 
import torch 
import os 
import torchvision 
import sys
sys.path.insert(1, 'C:\EECS504CNN\skin-classification-cv')
from skin_classification_cv.models.vgg import SkinVGG
from skin_classification_cv.loaders.imagenet_loader import ImageNetDataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 7)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x
   
       
    



    


In [9]:
import os
os.chdir('C:\EECS504CNN\skin-classification-cv')
dataset = ImageNetDataset('datasets/HAM_10000')

# Indexing an element
img, label = dataset[0]  # Get the first image and label



torch.manual_seed(42)
generator = torch.Generator().manual_seed(42)
train, val, test = torch.utils.data.random_split(dataset, [0.7, 0.2, 0.1], generator=generator)



batch_size = 32
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size)
test_loader = DataLoader(val, batch_size=batch_size)



# Initialize the model
model = SimpleCNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
    


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=7, bias=True)
)

In [12]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), (labels).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), torch.tensor(labels).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_corrects.double() / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Save the trained model
torch.save(model.state_dict(), 'custom_cnn_model2.pth')


C:\Users\johnd\AppData\Local\Temp\ipykernel_15840\4033755252.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images, labels = images.to(device), torch.tensor(labels).to(device)


Epoch 1/10, Train Loss: 0.9024, Val Loss: 0.7423, Val Accuracy: 0.7189
Epoch 2/10, Train Loss: 0.7381, Val Loss: 0.6973, Val Accuracy: 0.7349
Epoch 3/10, Train Loss: 0.6381, Val Loss: 0.7182, Val Accuracy: 0.7409
Epoch 4/10, Train Loss: 0.5055, Val Loss: 0.7390, Val Accuracy: 0.7454
Epoch 5/10, Train Loss: 0.3585, Val Loss: 0.9124, Val Accuracy: 0.7204
Epoch 6/10, Train Loss: 0.2306, Val Loss: 1.0755, Val Accuracy: 0.7179
Epoch 7/10, Train Loss: 0.1435, Val Loss: 1.2102, Val Accuracy: 0.7244
Epoch 8/10, Train Loss: 0.0784, Val Loss: 1.5187, Val Accuracy: 0.7294
Epoch 9/10, Train Loss: 0.0527, Val Loss: 1.6678, Val Accuracy: 0.7324
Epoch 10/10, Train Loss: 0.0794, Val Loss: 1.7391, Val Accuracy: 0.7224


In [14]:
model.eval()
val_loss = 0.0
val_corrects = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), torch.tensor(labels).to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        val_corrects += torch.sum(preds == labels.data)

val_accuracy = val_corrects.double() / len(test_loader.dataset)

print(f"Test Accuracy: {val_accuracy:.4f}")

C:\Users\johnd\AppData\Local\Temp\ipykernel_15840\487138890.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images, labels = images.to(device), torch.tensor(labels).to(device)


Test Loss: 1.7391, Test Accuracy: 0.7224
